In [1]:
import re
import glob
import pandas as pd
import sweetviz
import random
import pandas as pd
import warnings
import unidecode
import numpy as np
import csv
import emoji
from typing import List
from flair.data import Token
import sys,os
import tqdm
sys.path.append('../')

In [2]:
from preprocessing.amharicSegmenter import AmharicSegmenter

In [3]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [4]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [5]:
def remove_emoji(news):
    for idx,cnt in enumerate(news):
        news[idx] = re.sub(r'(:\S+)(@\w+)[A-Za-z]','', cnt)
    return news

In [6]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

Before Removing Duplicated Sentences
   Columns  Duplicate count
0  Content              280
After Removing Duplicated Sentences
   Columns  Duplicate count
0  Content                0


In [8]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)

# Read each CSV files and write it to text file

In [7]:
remove = ['\xa087','\xa0', '|', '/','…','«','"','“','”','»','-',
          '$','%','^','&','*','›','’','‘','‹','<','>','`','´','~','=','+']
# remove output file if it exists, then append each csv to a single file output
outfile = "all_sentences.txt"
os.remove(outfile) if os.path.exists(outfile) else None
# all csv files, to controll progress par
allnews = []
for news in glob.glob("data/*.csv"):
    allnews.append(news)

# read each files and write to a file system   
for news in tqdm.tqdm(allnews,  position=0, leave=True):
    data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
    data = data[data.Content.duplicated()==False].reset_index()
    data.Content = data.Content.apply(lambda x: remove_url(x))
    for token in remove:
        data.Content = data.Content.apply(lambda x: x.replace(token,' '))  
    text  = data.Content.to_string()
    with open (outfile,"a", encoding="utf-8") as all_sentences:
        for s in amseg.tokenize_sentence(text):
            all_sentences.write(s+'\n')


  0%|          | 0/4 [00:03<?, ?it/s]


NameError: name 'amseg' is not defined